In [1]:
import pdb
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib

In [2]:
bach = "https://imslp.org/index.php?title=Category:Bach%2C%20Johann%20Sebastian&customcat=ccperson1"
beethoven = "https://imslp.org/index.php?title=Category:Beethoven%2C%20Ludwig%20van&customcat=ccperson1"
mozart = "https://imslp.org/index.php?title=Category:Mozart%2C%20Wolfgang%20Amadeus&customcat=ccperson1"

In [3]:
def get_next_page(current_soup):
    
    links = current_soup.findAll('a', {"class":"categorypaginglink"})
    for link in links:
        if link.text == "next 200":
            return "https://imslp.org/" + link.get("href")
        
    return None

In [4]:
def format_work_df(df):
    df['link'] = df['Page Name'].apply(lambda x: "https://imslp.org/wiki/" + x.replace(" ", "_"))
    
    return df

def extract_composer_name(url):
    s = url.split("&")[0].split(":")[-1]
    return urllib.parse.unquote(s)
    
    return s

def get_work_pages(url):
    dfs = []
    composer = extract_composer_name(url)
    while url:
        html = requests.get(url).content
        table = pd.read_html(html)[0]
        dfs.append(table)
        
        soup = BeautifulSoup(html)
        url = get_next_page(soup)
        
    df = pd.concat(dfs)
    df['composer'] = composer
    
    return format_work_df(df).reset_index().drop("index", axis=1)
    

In [135]:
works_df = get_work_pages(mozart)

In [136]:
works_df

,Page Name,Cat. No.,I-Cat.,Key,Year,composer,link
0,"Abendempfindung, K.523 (Mozart, Wolfgang Amadeus)",K.523,IWM 1,F major,"1787 June 24, Vienna","Mozart, Wolfgang Amadeus","https://imslp.org/wiki/Abendempfindung,_K.523_..."
1,"Abendruhe, K.Anh.C 9.10 (Mozart, Wolfgang Amad...",K.Anh.C 9.10?,IWM 2,B-flat major,-,"Mozart, Wolfgang Amadeus","https://imslp.org/wiki/Abendruhe,_K.Anh.C_9.10..."
2,"Adagio and Allegro in F minor, K.594 (Mozart, ...",K.594,IWM 3,F minor,1790,"Mozart, Wolfgang Amadeus",https://imslp.org/wiki/Adagio_and_Allegro_in_F...
3,"Adagio and Fugue in C minor, K.546 (Mozart, Wo...",K.546,IWM 4,C minor,1788,"Mozart, Wolfgang Amadeus",https://imslp.org/wiki/Adagio_and_Fugue_in_C_m...
4,"Adagio and Menuett, K.Anh.34 (Mozart, Wolfgang...",K.Anh.34,IWM 5,D minor / D major,-,"Mozart, Wolfgang Amadeus","https://imslp.org/wiki/Adagio_and_Menuett,_K.A..."
5,"Adagio and Rondo in C minor, K.617 (Mozart, Wo...",K.617,IWM 6,C minor,1791,"Mozart, Wolfgang Amadeus",https://imslp.org/wiki/Adagio_and_Rondo_in_C_m...
6,"Adagio in B minor, K.540 (Mozart, Wolfgang Ama...",K.540,IWM 7,B minor,"1788-3-19, Vienna","Mozart, Wolfgang Amadeus","https://imslp.org/wiki/Adagio_in_B_minor,_K.54..."
7,"Adagio in B minor, K.deest (Mozart, Wolfgang A...",K.deest,IWM 8,B minor,1788,"Mozart, Wolfgang Amadeus","https://imslp.org/wiki/Adagio_in_B_minor,_K.de..."
8,"Adagio in B-flat major, K.411/484a (Mozart, Wo...",K.411 ; K⁶.484a,IWM 9,B-flat major,1783–84,"Mozart, Wolfgang Amadeus","https://imslp.org/wiki/Adagio_in_B-flat_major,..."
9,"Adagio in C major, K.356/617a (Mozart, Wolfgan...",K.356/617a,IWM 10,C major,1791,"Mozart, Wolfgang Amadeus","https://imslp.org/wiki/Adagio_in_C_major,_K.35..."
